<a href="https://colab.research.google.com/github/szkics/GerLangReconstruct/blob/main/machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install deep-translator

In [76]:
from deep_translator import GoogleTranslator

english_sentence = 'Das Meer ist ein kalter Ort.'
german_sentence = GoogleTranslator(source='german', target='dutch').translate(english_sentence, return_all=False)
print(german_sentence)

The sea is a cold place.


In [55]:
import nltk
from nltk import tokenize
nltk.download('punkt')

list_of_german_sentences = []
file = open('/content/gdrive/MyDrive/gerlangreconstruct/german_tales.txt','r').read()
list_of_german_sentences = tokenize.sent_tokenize(file)
print(len(list_of_german_sentences))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
3066


In [68]:
import regex as re
count = 0
german_sent_list = []
for sentence in list_of_german_sentences:
  processed_sentence = re.sub(r"\n", " ", sentence)
  number_of_words = len(processed_sentence.split())
  if (number_of_words <= 16):
    count += 1
    german_sent_list.append(processed_sentence)
count

1519

In [77]:
dutch_sent_list = []
english_sent_list = []
for german_sentence in l_german_sentences[:100]:
  english_sentence = GoogleTranslator(source='german', target='english').translate(german_sentence, return_all=False)
  english_sent_list.append(english_sentence) 
  dutch_sentence = GoogleTranslator(source='english', target='dutch').translate(english_sentence, return_all=False)
  dutch_sent_list.append(dutch_sentence)


In [78]:
german_sent_list[13]

'„Koax, Koax, breckekekex,“ war alles, was er sagen konnte, als er das hübsche, kleine Mädchen sah.'

In [79]:
dutch_sent_list[13]

"'Coax, coax, breckekekex,' was alles wat hij kon zeggen toen hij het mooie meisje zag."

In [80]:
english_sent_list[13]

'"Coax, coax, breckekekex," was all he could say when he saw the pretty little girl.'

In [81]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, Bidirectional
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [83]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [84]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [85]:
english_sentences = [clean_sentence(tmp) for tmp in english_sent_list]
dutch_sentences = [clean_sentence(tmp) for tmp in dutch_sent_list]
german_sentences = [clean_sentence(tmp) for tmp in l_german_sentences[:100]]

In [86]:
# Tokenize words
english_text_tokenized, english_text_tokenizer = tokenize(english_sentences)
german_text_tokenized, german_text_tokenizer = tokenize(german_sentences)
dutch_text_tokenized, dutch_text_tokenizer = tokenize(dutch_sentences)

print(english_text_tokenized)
print(english_text_tokenizer)

# Check language length
english_vocab = len(english_text_tokenizer.word_index) + 1
print("English vocabulary is of {} unique words".format(english_vocab))

german_vocab = len(german_text_tokenizer.word_index) + 1
print("german vocabulary is of {} unique words".format(german_vocab))

dutch_vocab = len(dutch_text_tokenizer.word_index) + 1
print("dutch vocabulary is of {} unique words".format(dutch_vocab))

[[152, 45, 1, 153, 87, 29, 154, 155], [156, 87], [88, 89, 90, 91, 92, 90, 91, 157, 93], [158, 29, 93, 159, 15, 92], [160, 161, 162], [163, 164], [94, 95, 9, 165, 21, 166, 6, 1, 167], [4, 5, 35, 96, 97, 10, 168, 169, 59, 170, 4, 5, 60, 11], [7, 171, 15, 4, 24, 98, 19, 172, 20, 1, 173, 174, 1, 46], [7, 36, 61, 99, 175, 176, 100, 177], [9, 101, 178, 62], [7, 30, 179, 180, 94, 31, 181, 2, 182, 102, 25, 36, 63, 183, 184], [185, 12, 5, 103, 31, 104, 2, 186, 1, 64, 187, 16, 65, 47], [105, 105, 188, 5, 13, 12, 30, 189, 48, 12, 190, 1, 191, 26, 106], [32, 7, 192, 22, 1, 104, 193, 37, 3, 1, 194, 195, 11, 5, 196], [1, 197, 107, 20, 1, 198, 2, 4, 107, 199, 200, 201], [202, 66, 203, 108, 11, 5, 48, 1, 204, 33, 38, 22, 17, 38, 1, 109], [205, 12, 206, 110, 111, 12, 67, 14, 3, 207, 3, 208], [18, 33, 112, 22, 17, 45, 1, 109, 2, 209, 17, 20, 10, 210], [211, 1, 113, 11, 212, 13, 213, 15, 1, 114, 115], [31, 113, 2, 49, 214, 19, 50, 27, 51, 1, 68, 116, 51], [32, 7, 27, 3, 1, 52, 215, 216], [69, 64, 217, 21

In [87]:
max_sentence_length = 16
english_pad_sentence = pad_sequences(english_text_tokenized, max_sentence_length, padding = "post")
german_pad_sentence = pad_sequences(german_text_tokenized, max_sentence_length, padding = "post")
dutch_pad_sentence = pad_sequences(dutch_text_tokenized, max_sentence_length, padding = "post")

# Reshape data
english_pad_sentence = english_pad_sentence.reshape(*english_pad_sentence.shape, 1)
german_pad_sentence = german_pad_sentence.reshape(*german_pad_sentence.shape, 1)
dutch_pad_sentence = dutch_pad_sentence.reshape(*dutch_pad_sentence.shape, 1)

In [131]:
input_shape = (max_sentence_length, 2)
input_sequence = Input(input_shape, name='InputLayer')
rnn = LSTM(512, return_sequences=True, dropout=0.1, name='RNNLayer')(input_sequence)
logits = TimeDistributed(Dense(german_vocab), name='TimeDistributed')(rnn)

model = Model(input_sequence, Activation('softmax')(logits))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-2),
              metrics=['accuracy'])

In [132]:
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (InputLayer)      [(None, 16, 2)]           0         
_________________________________________________________________
RNNLayer (LSTM)              (None, 16, 512)           1054720   
_________________________________________________________________
TimeDistributed (TimeDistrib (None, 16, 475)           243675    
_________________________________________________________________
activation_7 (Activation)    (None, 16, 475)           0         
Total params: 1,298,395
Trainable params: 1,298,395
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(english_pad_sentence[0].shape)
print(english_pad_sentence[0])

print(dutch_pad_sentence[0].shape)
print(dutch_pad_sentence[0])

In [ ]:
english_dutch_data = np.array([np.concatenate((english_pad_sentence[i], dutch_pad_sentence[i]), axis=1) for i in range(len(english_pad_sentence))])
print(english_dutch_data.shape)
print(english_dutch_data[0])

In [135]:
model_results = model.fit(english_dutch_data, german_pad_sentence, batch_size=10, epochs=200)

Epoch 1/200
10/10 [==============================] - 3s 127ms/step - loss: 4.7297 - accuracy: 0.3034
Epoch 2/200
10/10 [==============================] - 1s 128ms/step - loss: 3.8947 - accuracy: 0.3409
Epoch 3/200
10/10 [==============================] - 1s 134ms/step - loss: 3.5371 - accuracy: 0.3751
Epoch 4/200
10/10 [==============================] - 1s 134ms/step - loss: 3.3659 - accuracy: 0.3800
Epoch 5/200
10/10 [==============================] - 1s 130ms/step - loss: 3.0184 - accuracy: 0.4316
Epoch 6/200
10/10 [==============================] - 1s 125ms/step - loss: 2.8193 - accuracy: 0.4503
Epoch 7/200
10/10 [==============================] - 1s 122ms/step - loss: 2.8672 - accuracy: 0.4213
Epoch 8/200
10/10 [==============================] - 1s 123ms/step - loss: 2.6096 - accuracy: 0.4587
Epoch 9/200
10/10 [==============================] - 1s 127ms/step - loss: 2.4483 - accuracy: 0.4840
Epoch 10/200
10/10 [==============================] - 1s 130ms/step - loss: 2.4272 - accura

In [93]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [99]:
index = 60
print("The english sentence is: {}".format(english_sentences[index]))
print("The dutch sentence is: {}".format(dutch_sentences[index]))
# AFTER 1 EPOCH
print("The actual german sentence is    : {}".format(german_sentences[index]))
print('The predicted german sentence is: ', end="")
print(logits_to_sentence(model.predict(english_dutch_data[index:index+1])[0], german_text_tokenizer))

The english sentence is: they were fastened to thumbelies back and now she too could fly from flower to flower
The dutch sentence is: ze werden vastgemaakt aan duimelijntjes rug en nu kon ook zij van bloem naar bloem vliegen
The actual german sentence is    : sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen
The predicted german sentence is: <empty> die nicht die nicht nicht <empty> nicht nicht nicht die als nicht als er nicht


In [101]:
# AFTER 10 EPOCHS
print("The actual german sentence is    : {}".format(german_sentences[index]))
print('The predicted german sentence is: ', end="")
print(logits_to_sentence(model.predict(english_dutch_data[index:index+1])[0], german_text_tokenizer))

sie däumelieschen sie däumelieschen däumelieschen däumelieschen und sagte die <empty> <empty> <empty> <empty> <empty> <empty> <empty>


In [103]:
# AFTER 100 EPOCHS
print("The actual german sentence is    : {}".format(german_sentences[index]))
print('The predicted german sentence is: ', end="")
print(logits_to_sentence(model.predict(english_dutch_data[index:index+1])[0], german_text_tokenizer))

The actual german sentence is    : sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen
The predicted german sentence is: sie blickte rücken größer rücken niedlich und auf er auch sie sie wiesen der konnte nie


In [110]:
# AFTER 100 EPOCHS AND CHANGING NUMBER OF HIDDEN NODES TO 512 AND DROPOUT TO 0.2
print("The actual german sentence is    : {}".format(german_sentences[index]))
print('The predicted german sentence is: ', end="")
print(logits_to_sentence(model.predict(english_dutch_data[index:index+1])[0], german_text_tokenizer))

The actual german sentence is    : sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen
The predicted german sentence is: sie wurden rücken am rücken befestigt und nun konnte auch sie von blume zu blume fliegen


In [124]:
# AFTER 100 EPOCHS AND CHANGING DROPOUT TO 0.1 (BEST MODEL YET FOR THIS DATA, 512 + 0.1)
print("The actual german sentence is    : {}".format(german_sentences[index]))
print('The predicted german sentence is: ', end="")
print(logits_to_sentence(model.predict(english_dutch_data[index:index+1])[0], german_text_tokenizer))

The actual german sentence is    : sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen
The predicted german sentence is: sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen


In [130]:
# 100 EPOCHS, 1024, 0.1
print("The actual german sentence is    : {}".format(german_sentences[index]))
print('The predicted german sentence is: ', end="")
print(logits_to_sentence(model.predict(english_dutch_data[index:index+1])[0], german_text_tokenizer))

The actual german sentence is    : sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen
The predicted german sentence is: sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen


In [136]:
# 200 EPOCHS, 512, 0.1
print("The actual german sentence is    : {}".format(german_sentences[index]))
print('The predicted german sentence is: ', end="")
print(logits_to_sentence(model.predict(english_dutch_data[index:index+1])[0], german_text_tokenizer))

The actual german sentence is    : sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen
The predicted german sentence is: WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f5afe395560> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
sie wurden däumelieschen am rücken befestigt und nun konnte auch sie von blume zu blume fliegen


In [144]:
english_test_sentence = clean_sentence("The little girl was lost in the forest and suddenly a wolf appeared.")
dutch_test_sentence = clean_sentence("Het kleine meisje was verdwaald in het bos en plotseling verscheen er een wolf.")
expected_german_sentence = clean_sentence("Das kleine Mädchen verirrte sich im Wald und plötzlich tauchte ein Wolf auf.")

In [148]:
english_sentences.append(english_test_sentence)
dutch_sentences.append(dutch_test_sentence)

In [149]:
english_text_tokenized, english_text_tokenizer = tokenize(english_sentences)
dutch_text_tokenized, dutch_text_tokenizer = tokenize(dutch_sentences)

In [150]:
english_pad_sentence = english_pad_sentence.reshape(*english_pad_sentence.shape, 1)
dutch_pad_sentence = dutch_pad_sentence.reshape(*dutch_pad_sentence.shape, 1)

In [ ]:
english_dutch_data = np.array([np.concatenate((english_pad_sentence[i], dutch_pad_sentence[i]), axis=1) for i in range(len(english_pad_sentence))])
print(english_dutch_data.shape)
print(english_dutch_data[99])

In [156]:
index = 99
print("The english sentence is: {}".format(english_sentences[index]))
print("The dutch sentence is: {}".format(dutch_sentences[index]))
print("The actual german sentence is    : {}".format(german_sentences[index]))
print('The predicted german sentence is: ', end="")
print(logits_to_sentence(model.predict(english_dutch_data[index:index+1])[0], german_text_tokenizer))

The english sentence is: since then all storks have been called peter and are still called that today
The dutch sentence is: sindsdien heten alle ooievaars peter en worden ze nu nog steeds zo genoemd
The actual german sentence is    : seitdem hießen alle störche peter und werden noch heute so genannt
The predicted german sentence is: „laßt hießen alle störche peter und werden noch heute so genannt <empty> <empty> <empty> <empty> <empty>
